# Tasks

> FrankenUI Tasks Example

In [36]:
#| default_exp tasks

In [37]:
#| hide
from nbdev.showdoc import *

In [38]:
#| export
from fasthtml.common import *
import fasthtml.common as fh
from fh_frankenui import *
from fasthtml.svg import *
import json

In [39]:
%%html
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">

<style>
#notebook-container { max-width: none; }
.output_html * { list-style-type: none !important; }
</style>

## Data

We store data in Python to minic the static data in the examples.  Eventually this should be replaced with sqlite data store to allow for interactive functionality

In [40]:
#| export
with open('data/status_list.json', 'r') as f: data     = json.load(f)
with open('data/statuses.json',    'r') as f: statuses = json.load(f)

In [41]:
#| export
priority_dd = [{'priority': "low", 'count': 36 }, {'priority': "medium", 'count': 33 }, {'priority': "high", 'count': 31 }]

rows_per_page_dd = [10,20,30,40,50]

status_dd = [{'status': "backlog", 'count': 21 },{'status': "todo", 'count': 21 },{'status': "progress", 'count': 20 },{'status': "done",'count': 19 },{'status': "cancelled", 'count': 19 }]


## Tasks Page Functions

In [42]:
#| export
def create_hotkey_li(hotkey):
    return Li(A(cls='uk-drop-close justify-between')(
        hotkey[0],
        Span(hotkey[1], cls='ml-auto text-xs tracking-widest opacity-60')))

hotkeys_a = (('Profile','⇧⌘P'),('Billing','⌘B'),('Settings','⌘S'),('New Team',''))
hotkeys_b = (('Logout',''), )

# TODO convert to navbar
avatar_opts = Ul(cls='uk-dropdown-nav uk-nav')(
    Li(cls='px-2 py-1.5 text-sm')(
        Div(cls='flex flex-col space-y-1')(
            P('sveltecult', cls='text-sm font-medium leading-none'),
            P('leader@sveltecult.com', cls='text-xs leading-none text-muted-foreground'))),
    Li(cls='uk-nav-divider'),
    *map(create_hotkey_li, hotkeys_a),
    Li(cls='uk-nav-divider'),
    *map(create_hotkey_li, hotkeys_b)
)

In [43]:
#| export
def CreateTaskModal():
    return Modal(
        Div(cls='p-6')(
            ModalTitle('Create Task'),P('Fill out the information below to create a new task', cls=TextFont.muted_sm),
            Br(),
            Form(cls='space-y-6')(
                Grid(Div(Select(*Options('Documentation', 'Bug', 'Feature'), label='Task Type', id='task_type')),
                     Div(Select(*Options('In Progress', 'Backlog', 'Todo', 'Cancelled', 'Done'), label='Status', id='task_status')),
                     Div(Select(*Options('Low', 'Medium', 'High'), label='Priority', id='task_priority'))),
                TextArea(label='Title', placeholder='Please describe the task that needs to be completed'),
                Div(cls='flex justify-end space-x-2')(
                    ModalCloseButton('Cancel', cls=ButtonT.ghost),
                    ModalCloseButton('Submit', cls=ButtonT.primary),))),
        id='TaskForm')

In [73]:
#| export
page_heading =SpaceBetweenDiv(cls='space-y-2')(
            Div(cls='space-y-2')(
                H2('Welcome back!'),P("Here's a list of your tasks for this month!", cls=TextFont.muted_sm)),
            Div(A(href='#', cls='h-8 w-8 inline-flex rounded-full bg-accent ring-ring')(fh.Img(src='https://api.dicebear.com/8.x/lorelei/svg?seed=sveltecult')),
                Div(uk_dropdown='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop')(avatar_opts)))

In [74]:
show(page_heading)

In [26]:
#| export
table_controls =(Input(cls='w-[250px]',placeholder='Filter task'),
     UkDropdownButton(
         [A(FullySpacedDiv(P(a['status']), P(a['count'])),cls=TextT.capitalize) for a in status_dd],
         label = "Status", 
         btn_cls=(TextFont.bold_sm, ButtonT.default)),     
     UkDropdownButton(
         [A(FullySpacedDiv(LAlignedIconTxt(a['priority'], icon="check"), a['count']),cls=TextT.capitalize) for a in priority_dd],                         
         label = "Priority", 
         btn_cls=(TextFont.bold_sm,ButtonT.default)),
      UkDropdownButton([A(LAlignedIconTxt(o, icon="check")) for o in ['Title','Status','Priority']],
            label='View',
            opt_hdrs=["Toggle Columns"],
            btn_cls=(TextFont.bold_sm,ButtonT.default)),
            Button('Create Task',cls=(ButtonT.primary, TextFont.bold_sm), uk_toggle="target: #TaskForm"))

In [27]:
Button('Create Task',cls=(ButtonT.primary, TextFont.bold_sm), uk_toggle="target: #TaskForm")

```html
<button type="button" uk-toggle="target: #TaskForm" class="uk-button uk-button-primary uk-text-bold uk-text-small">Create Task</button>
```

## Route Data

In [11]:
#| export
def task_dropdown():
    # TODO: Figure out uk-drop-close boilerplate fix
    return UkDropdownButton([
            A('Edit',                   cls='uk-drop-close'),
            A('Make a copy',            cls='uk-drop-close'),
            A('Favorite',               cls='uk-drop-close')],
            [A(SpacedPP('Delete', '⌘⌫'), cls='uk-drop-close')])

In [12]:
#| export
def header_render(col):
    cls = 'p-2 ' + 'uk-table-shrink' if col in ('Done','Actions') else ''
    match col:
        case "Done":    return Th(CheckboxX(), cls=cls)
        case 'Actions': return Th("",       cls=cls)
        case _:         return Th(col, cls=cls)

In [82]:
#| export
def cell_render(col, row):
    def _Td(*args,cls='', **kwargs): return Td(*args, cls=f'p-2 {cls}',**kwargs)
    match col:
        case "Done":  return _Td(shrink=True)(CheckboxX(selected=row['selected']))
        case "Task":  return _Td(row["id"])
        case "Title": return _Td(cls='max-w-[500px] truncate', expand=True)(row["title"], cls='font-medium')
        case "Status" | "Priority": return _Td(cls='uk-text-nowrap uk-text-capitalize')(Span(row[col.lower()]))
        case "Actions": return _Td(cls='uk-table-shrink')(task_dropdown())
        case _: raise ValueError(f"Unknown column: {col}")

In [48]:
#| export
task_columns = ["Done", 'Task', 'Title', 'Status', 'Priority', 'Actions']

tasks_table = Div(cls='uk-overflow-auto mt-4 rounded-md border border-border')(TableFromDicts(
    header_data=task_columns,
    body_data=data,
    body_cell_render=cell_render,
    header_cell_render=header_render))
# show(tasks_table)

In [49]:
#| export

def footer():
    hw_cls = 'h-4 w-4'
    return SpaceBetweenDiv(cls='mt-4 px-2 py-2')(
        Div('1 of 100 row(s) selected.', cls='flex-1 text-sm text-muted-foreground'),
        Div(cls='flex flex-none items-center space-x-8')(
            CenteredDiv('Page 1 of 10', cls='w-[100px] text-sm font-medium'),
            LAlignedDiv(
                UkIconButton(cls='hidden lg:inline-flex')(Span('Go to last page', cls='sr-only'),
                    Span(uk_icon='chevron-double-left', cls=hw_cls)),
                UkIconButton(Span('Go to previous page', cls='sr-only'),
                    Span(uk_icon='chevron-left', cls=hw_cls)),
                UkIconButton(Span('Go to next page', cls='sr-only'),
                    Span(uk_icon='chevron-right', cls=hw_cls)),
                UkIconButton(cls='hidden lg:inline-flex')(
                    Span('Go to last page', cls='sr-only'),
                    Span(uk_icon='chevron-double-right', cls=hw_cls)),
            gap=2)))

In [50]:
#| export
tasks_ui = Div(
    SpaceBetweenDiv(cls='mt-8')(
        Div(cls='flex flex-1 gap-4')(table_controls)),
    tasks_table,
    footer(),)

In [ ]:
#| export
tasks_homepage = CreateTaskModal(), Div(cls='p-8')(page_heading, tasks_ui)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()